In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\nlp-getting-started\train.csv")
dftest = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\nlp-getting-started\test.csv")

In [3]:
train = df[:7000]
validation = df[7000:]

In [4]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
6995,10031,twister,NaN,@ellenfromnowon 7-speed nexus shifter å£9! (Fo...,0
6996,10032,twister,"Calgary, Alberta",Anyone wanna come over and watch Twister with ...,0
6997,10034,twister,NY,Crazy Mom Threw Teen Daughter a NUDE Twister S...,1
6998,10035,twister,"Plano,TX",@briannafrost Twister with Bill Paxton and Hel...,0


In [5]:
validation

,id,keyword,location,text,target
7000,10037,twister,"Midwest City, OK",HAPPY 24 TWISTER!!! Thank you for all the laug...,0
7001,10038,twister,NaN,'How many men would a human hew if a human cou...,0
7002,10039,twister,NaN,I liked a @YouTube video http://t.co/Cj76K0YaY...,0
7003,10040,twister,NaN,Tornadoes: El Nino may give Canada's twister s...,1
7004,10041,twister,"Lisbon, Portugal",@ElianaRaquel Like GG was BAD in the end... Bu...,0
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
dftest

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [7]:
class preprocess():
    def __init__(self,bigdataorig,train,test = False, validation = False):
        self.train = train
        self.test = test
        self.validation = validation
        self.bigdataorig = bigdataorig
        
    def transform(self):
        if self.test == False and self.validation == False:
            zero_train = self.train[self.train['target'] == 0]
            one_train = self.train[self.train['target'] == 1]
            for i in zero_train.index:
                if len(zero_train) != len(one_train):
                    zero_train = zero_train.drop(i,axis = 0)
            dataset = pd.concat([one_train,zero_train])
            
            import re
            u = []
            for i in dataset['text']:
                cleaned = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split())
                cleaned= cleaned.lower()
                u.append(cleaned)
            dataset['text'] = u
            listof = list(self.bigdataorig.keyword.unique())[1:]
            v= []
            for i in [i for i in listof if '%20' in i]:
                v.append((i.split('%20')))

            for i in v:
                for u in i:
                    listof.append(u)   
            a = []
            for k in dataset.index:
                if [i for i in dataset['text'][k].split() if i in listof]:
                    a.append([i for i in dataset['text'][k].split() if i in listof][0])
                else:
                    a.append('fire')
            f = []
            c = 0
            for i in (dataset['keyword'].index):
                if type(dataset['keyword'][i]) != str:
                    f.append(a[c])
                else:
                    f.append(dataset['keyword'][i])
                c+=1
            dataset['keyword'] = f
            dataset = dataset.drop(['location'],axis = 1)
            dataset = dataset.sample(frac=1).reset_index(drop=True)
            aa = []
            for i in dataset['keyword']:
                if "%20" in i:
                    aa.append(i.replace('%20'," "))
                else:
                    aa.append(i)
            dataset['keyword'] = aa
            return dataset
        
        if self.test == False and self.validation == True:
            import re
            u = []
            for i in self.train['text']:
                cleaned = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split())
                cleaned= cleaned.lower()
                u.append(cleaned)
            self.train['text'] = u
            listof = list(self.bigdataorig.keyword.unique())[1:]
            v= []
            for i in [i for i in listof if '%20' in i]:
                v.append((i.split('%20')))

            for i in v:
                for u in i:
                    listof.append(u)   
            a = []
            for k in self.train.index:
                if [i for i in self.train['text'][k].split() if i in listof]:
                    a.append([i for i in self.train['text'][k].split() if i in listof][0])
                else:
                    a.append('fire')
            f = []
            c =0
            for i in self.train['keyword'].index:
                if type(self.train['keyword'][i]) != str:
                    f.append(a[c])
                else:
                    f.append(self.train['keyword'][i])
                c+=1
            self.train['keyword'] = f
            self.train = self.train.drop(['location'],axis = 1)
            aa = []
            for i in self.train['keyword']:
                if "%20" in i:
                    aa.append(i.replace('%20'," "))
                else:
                    aa.append(i)
            self.train['keyword'] = aa
            return self.train
        
        if self.test == True and self.validation == False:
            import re
            u = []
            for i in self.train['text']:
                cleaned = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split())
                cleaned= cleaned.lower()
                u.append(cleaned)
            self.train['text'] = u
            listof = list(self.bigdataorig.keyword.unique())[1:]
            v= []
            for i in [i for i in listof if '%20' in i]:
                v.append((i.split('%20')))

            for i in v:
                for u in i:
                    listof.append(u)   
            a = []
            for k in self.train.index:
                if [i for i in self.train['text'][k].split() if i in listof]:
                    a.append([i for i in self.train['text'][k].split() if i in listof][0])
                else:
                    a.append('fire')
            f = []
            c =0
            for i in self.train['keyword'].index:
                if type(self.train['keyword'][i]) != str:
                    f.append(a[c])
                else:
                    f.append(self.train['keyword'][i])
                c+=1
            self.train['keyword'] = f
            self.train = self.train.drop(['location'],axis = 1)
            aa = []
            for i in self.train['keyword']:
                if "%20" in i:
                    aa.append(i.replace('%20'," "))
                else:
                    aa.append(i)
            self.train['keyword'] = aa
            return self.train
            
                    
            

            

In [8]:
p = preprocess(df,train)
newtrain = p.transform()

v = preprocess(df,validation,validation = True)
newval = v.transform()

y = preprocess(df,dftest,test = True)
newtest = y.transform()

C:\Users\HP\AppData\Local\Temp/ipykernel_7676/262834611.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train['text'] = u
C:\Users\HP\AppData\Local\Temp/ipykernel_7676/262834611.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train['keyword'] = f


In [9]:
idd = newtest.id
newtest = newtest.drop(['id'],axis = 1)

In [10]:
newtrainy = newtrain.target
newtrainX = newtrain.drop(['target','id'],axis = 1)

In [11]:
newtrainX

,keyword,text
0,mudslide,oso washington mudslide response interview part 1
1,survivors,as anniversary nears atomic bomb survivors spe...
2,oil spill,dtn brazil refugio oil spill may have been cos...
3,seismic,u control the future of india yor subject expl...
4,landslide,this govt of hubris has small maj yet acts as ...
...,...,...
5889,smoke,im gonna get naked and smoke my cigarette some...
5890,hailstorm,well so much for outdoor postering mothernatur...
5891,smoke,and all you girls that smoke have you notice t...
5892,floods,gt as soon as maintenance ends everyone floods...


In [12]:
newval
newvaly = newval.target
newvalX = newval.drop(['target','id'],axis = 1)

In [13]:
newvalX

,keyword,text
7000,twister,happy 24 twister thank you for all the laughs ...
7001,twister,how many men would a human hew if a human coul...
7002,twister,i liked a video extreme paint twister
7003,twister,tornadoes el nino may give canada s twister se...
7004,twister,like gg was bad in the end but at least i care...
...,...,...
7608,bridge,two giant cranes holding a bridge collapse int...
7609,wild,ahrary the out of control wild fires in califo...
7610,volcano,m1 94 01 04 utc 5km s of volcano hawaii
7611,police,police investigating after an e bike collided ...


In [14]:
final = pd.concat([newtrainX,newvalX,newtest])

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_traincv = cv.fit_transform(final['text'])
Xt = cv.fit_transform(final['keyword'])
nnnx = X_traincv.toarray()
nnnxx = Xt.toarray()
n = pd.DataFrame(nnnx)
x = pd.DataFrame(nnnxx)
df_concat = pd.concat([x, n], axis=1)


In [16]:
df_concat

,0,1,2,3,4,5,6,7,8,9,...,17091,17092,17093,17094,17095,17096,17097,17098,17099,17100
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9765,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9766,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9767,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
train = df_concat[:5894]
val = df_concat[5894:5894+613]
test = df_concat[5894+613:]

In [18]:
train

,0,1,2,3,4,5,6,7,8,9,...,17091,17092,17093,17094,17095,17096,17097,17098,17099,17100
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5890,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(train, newtrainy, test_size=0.23, random_state=42)
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
loss = ['hinge', 'log'] 
penalty = ['l1', 'l2'] 
alpha = [0.0001, 0.001, 0.01, 0.1] 
learning_rate = ['optimal', 'invscaling', 'adaptive'] 
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}] 
eta0 = [1, 10] 
param_grid =dict(loss=loss, penalty=penalty, alpha=alpha, learning_rate=learning_rate, class_weight=class_weight, eta0=eta0) 
clf = SGDClassifier(max_iter = 1000)

grid = GridSearchCV(estimator = clf, param_grid = param_grid, cv=5)




In [20]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SGDClassifier(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'class_weight': [{0: 0.5, 1: 0.5}, {0: 0.6, 1: 0.4}],
                         'eta0': [1, 10],
                         'learning_rate': ['optimal', 'invscaling', 'adaptive'],
                         'loss': ['hinge', 'log'], 'penalty': ['l1', 'l2']})

In [21]:
best = grid.best_estimator_

In [22]:
print(f'the best parameters are {grid.best_params_}, and the best score is {grid.best_score_}')

the best parameters are {'alpha': 0.0001, 'class_weight': {1: 0.4, 0: 0.6}, 'eta0': 1, 'learning_rate': 'invscaling', 'loss': 'hinge', 'penalty': 'l2'}, and the best score is 0.8133613257629111


In [23]:
best

SGDClassifier(class_weight={0: 0.6, 1: 0.4}, eta0=1, learning_rate='invscaling')

In [24]:
predicted_score = best.predict(X_test)
accuracy = f'{100*best.score(X_test,y_test)}% Accuracy'

In [25]:
accuracyv = f'{100*best.score(val,newvaly)}% Accuracy'

In [26]:
accuracy,accuracyv

('79.20353982300885% Accuracy', '79.7716150081566% Accuracy')

In [27]:
predicted_scoretest = best.predict(test)

In [28]:
submit = pd.DataFrame()
submit['id'] = idd
submit['target'] = predicted_scoretest

In [29]:
submit

,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [30]:
submit.to_csv('nlpgrid.csv',index = None)